In [1]:
from PIL import Image
import pygsp.graphs as graphs
import numpy as np
import matplotlib.pyplot as plt
import spectral_denoiser
import importlib
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Comparison to Local Averaging

In [2]:
N = 80
True_Image = np.asarray(Image.open("noodle.jpg").resize((N,N)))
G = graphs.Grid2d(N)

In [3]:
Denoising_Machine = spectral_denoiser.Spectral_Denoiser(G)

In [4]:
df = pd.DataFrame()

In [5]:
spectral_errors = []
k_nn_errors = []

for i,sigma in enumerate([25,50,75,100]):
    A = G.W.todense()
    P = np.diag(1/np.array(np.sum(A,axis=0))[0])@A

    Noisy_Image = True_Image + np.random.randn(N,N,3)*sigma
    Noisy_Reshaped = Noisy_Image.reshape(-1,3)
    Denoised_Image = Denoising_Machine.remove_gaussian_noise(Noisy_Reshaped)
    Denoised_Image_Reshaped = Denoised_Image.reshape(N,N,3)
    
    se = np.linalg.norm(Denoised_Image_Reshaped - True_Image)
    spectral_errors.append(se)
    Neighbor_Avgs = Noisy_Reshaped
    
    k_nn_sigma = []
    for k in range(4):
        Neighbor_Avgs = P@(P@(P@Neighbor_Avgs))
        reshaped = np.array(Neighbor_Avgs).reshape(N,N,3)
        ke = np.linalg.norm(reshaped - True_Image)
        k_nn_sigma.append(ke)
    k_nn_errors.append(k_nn_sigma)
    

2023-04-26 09:41:02,559:[WARNING](pygsp.graphs.graph.lmax): The largest eigenvalue G.lmax is not available, we need to estimate it. Explicitly call G.estimate_lmax() or G.compute_fourier_basis() once beforehand to suppress the warning.


In [6]:
df['Spectral'] = spectral_errors
df['K=3'] = np.array(k_nn_errors)[:,0]
df['K=6'] = np.array(k_nn_errors)[:,1]
df['K=9'] = np.array(k_nn_errors)[:,2]
df['K=12'] = np.array(k_nn_errors)[:,3]
df.index = ['sigma = 25', 'sigma = 50', 'sigma = 75', 'sigma = 100']

In [7]:
df.head()

,Spectral,K=3,K=6,K=9,K=12
sigma = 25,2049.723901,1864.908968,2144.918350,2400.680087,2608.446799
sigma = 50,2380.880234,2643.729129,2519.843304,2632.970359,2771.356238
sigma = 75,2719.620206,3613.236103,3112.159172,3044.429945,3077.907374
sigma = 100,3060.564504,4631.744934,3778.815027,3538.984638,3473.834498
